### Imports

In [1]:
import pandas as pd

### Récuperation des données qui sont dans le CSV

In [21]:
datas = pd.read_csv('datas/datas.csv', delimiter='\t')

### Récupération de la date

In [22]:
headers = datas.columns.drop('Time')

datas['Day'] = datas.Time.apply(lambda x: x.split('-')[-1])
datas['Month'] = datas.Time.apply(lambda x: x.split('-')[1])
datas['Year'] = datas.Time.apply(lambda x: x.split('-')[0])

In [23]:
datas

,Time,"BTC / Price, USD","ETH / Price, USD","XRP / Price, USD","LINK / Price, USD","XLM / Price, USD","USDT / Price, USD","USDT_ETH / Price, USD","USDT_TRX / Price, USD","DASH / Price, USD",...,"WTC / Price, USD","XEM / Price, USD","XMR / Price, USD","XVG / Price, USD","XTZ / Price, USD","YFI / Price, USD","ZEC / Price, USD",Day,Month,Year
0,2017-03-01,1228.459651,17.265684,0.005330,NaN,0.001731,0.999667,NaN,NaN,43.279388,...,NaN,0.007874,12.377001,NaN,NaN,NaN,41.565084,01,03,2017
1,2017-03-02,1262.931467,19.204561,0.006133,NaN,0.001799,1.006446,NaN,NaN,42.546546,...,NaN,0.008749,13.213293,NaN,NaN,NaN,40.045525,02,03,2017
2,2017-03-03,1289.363191,19.654703,0.006466,NaN,0.001789,1.003244,NaN,NaN,47.777786,...,NaN,0.009055,14.029060,NaN,NaN,NaN,42.348426,03,03,2017
3,2017-03-04,1268.940664,18.684225,0.006247,NaN,0.001750,1.005842,NaN,NaN,43.735475,...,NaN,0.009835,13.808267,NaN,NaN,NaN,41.372994,04,03,2017
4,2017-03-05,1276.273683,19.301111,0.006151,NaN,0.001742,0.999961,NaN,NaN,42.373144,...,NaN,0.011450,15.263604,NaN,NaN,NaN,39.567362,05,03,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807,2022-02-10,43614.342696,3080.565482,0.834418,17.553471,0.232654,1.000671,1.000671,1.000671,111.891064,...,0.593469,0.114365,180.655497,0.012299,4.152122,24498.512741,127.082193,10,02,2022
1808,2022-02-11,42357.321742,2924.080468,0.759247,16.146358,0.215302,1.000517,1.000517,1.000517,103.997657,...,0.550347,0.108235,168.596810,0.011535,4.220464,23068.136834,116.299055,11,02,2022
1809,2022-02-12,42173.459396,2910.836175,0.819350,15.943319,0.217710,1.000562,1.000562,1.000562,103.734638,...,0.552108,0.108695,173.587783,0.011503,4.089775,23100.542446,117.484142,12,02,2022
1810,2022-02-13,42195.594745,2880.696526,0.809239,15.718898,0.211419,1.000460,1.000460,1.000460,102.548106,...,0.546389,0.110236,174.332235,0.011014,3.871521,22934.195769,117.312547,13,02,2022


In [37]:
liste_crypto_df = {}
liste_title = []
for crypto in headers:
    df = pd.DataFrame(datas[crypto])

    title = crypto.split(' ')[0]
    liste_title.append(title)
    liste_crypto_df[title] = df

In [35]:
liste_crypto_df['BTC']

,"BTC / Price, USD"
0,1228.459651
1,1262.931467
2,1289.363191
3,1268.940664
4,1276.273683
...,...
1807,43614.342696
1808,42357.321742
1809,42173.459396
1810,42195.594745
